In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
import pickle as pkl
import ast
import os
from sklearn.metrics.pairwise import cosine_similarity
from random import sample

In [2]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
LEVEL1_FOLDERS = ['groups']
allfiles = []

for l in LEVEL1_FOLDERS:
    l1path = '/data/shruti/ONR/big_data/facebook/' + l + '/*'
    LEVEL2_FOLDERS = glob.glob(l1path)
    for l2 in LEVEL2_FOLDERS:
        l2path = l2 + "/*.json"
        fnames = glob.glob(l2path)
        for f in fnames:
            if os.stat(f).st_size == 0:
                fakevar=1
            else:
                allfiles.append(f)


In [4]:
allfiles

['/data/shruti/ONR/big_data/facebook/groups/hindi/2021-01-01_2021-02-01.json',
 '/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-01-01_2021-02-01.json',
 '/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-02-01_2021-03-01.json',
 '/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-03-01_2021-04-01.json',
 '/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-04-01_2021-05-01.json',
 '/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-05-01_2021-06-01.json',
 '/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-06-01_2021-07-01.json',
 '/data/shruti/ONR/big_data/facebook/groups/re_hindi/2021-07-01_2021-08-01.json']

In [5]:
def get_contents(fileName):
    dictlist = []
    with open(fileName, "r") as jfile:
        for line in jfile:
            tempdict = {}
            try:
                job = json.loads(line)
            except:
                job = None
            if job!=None:    
                if "date" in job:
                    tempdict['date'] = job['date']
                    if "title" in job:
                        tempdict['title'] = job['title']
                    if "caption" in job:
                        tempdict['caption'] = job['caption']
                    if "description" in job:
                        tempdict['description'] = job['description']
                    if "description" in job:
                        tempdict['description'] = job['description']
                    if "message" in job:
                        tempdict['message'] = job['message']
                    if "link" in job:
                        tempdict['link'] = job['link']
                    if "postUrl" in job:
                        tempdict['postUrl'] = job['postUrl']
                    if "score" in job:
                        tempdict['score'] = job['score']

                    if "account" in job:
                        try:
                            tempdict['handle'] = job['account']['name']
                        except:
                            fakevar=1

                        try:
                            tempdict['platformId'] = job['account']['platformId']
                        except:
                            fakevar=1

                        try:
                            tempdict['id'] = job['account']['id']
                        except:
                            fakevar=1

                    dictlist.append(tempdict)

    if len(dictlist) > 0:
        return dictlist


In [6]:
filcontent = pd.DataFrame()
filcontent['fileNames'] = allfiles
filcontent['content'] = filcontent['fileNames'].parallel_apply(lambda x: get_contents(x))

In [7]:
jsonList = []

for idx, row in filcontent.iterrows():
    for c in row['content']:
        jsonList.append(c)

In [8]:
fdata = pd.DataFrame(jsonList)

In [9]:
fdata.head()

,date,postUrl,score,handle,platformId,id,message,title,description,link,caption
0,2021-01-31 23:59:57,https://www.facebook.com/groups/11349790768873...,-36.000000,🚩मिशन 2022 में योगी सरकार 🚩और 2024 में मोदी सर...,1134979076887377,14745293,NaN,NaN,NaN,NaN,NaN
1,2021-01-31 23:59:51,https://www.facebook.com/groups/22292669671263...,-14.000000,मैं मोदी और योगी आदित्यनाथ का सार्थक हूं,2229266967126315,14848753,Jay hind jay mahadev jay modi ji,NaN,NaN,NaN,NaN
2,2021-01-31 23:59:46,https://www.facebook.com/groups/74054456632282...,5.713333,हिंदुत्व सम्राट योगी आदित्यनाथ ✅,740544566322820,8951434,सुबह के 5:30 बज रहे हैं जो भी भाई बहन उठ गये ह...,NaN,NaN,NaN,NaN
3,2021-01-31 23:59:44,https://www.facebook.com/groups/27852592450850...,-1.052632,Jaya Kishori ji राधे राधे जी इस ग्रुप में अपने...,2785259245085048,14866321,NaN,NaN,NaN,NaN,NaN
4,2021-01-31 23:59:27,https://www.facebook.com/groups/29915130048167...,-26.000000,क्लस्टर और डीटीसी युवा एम्प्लॉई,299151300481673,14595402,NaN,This is a re-share of a post,विचारधारा कोई भी हो लेकिन अपनों को रोता नहीं छ...,https://www.facebook.com/100004617128135/posts...,NaN


In [10]:
print(len(fdata))
fdata = fdata.dropna(subset=['message'])
print(len(fdata))

4821747
2225610


In [11]:
fdata = fdata.drop_duplicates()
len(fdata)

1982899

In [12]:
fdata.sample(5)

,date,postUrl,score,handle,platformId,id,message,title,description,link,caption
2639521,2021-03-01 00:40:19,https://www.facebook.com/groups/80588631989662...,-36.000000,मेरे साई सरकार,805886319896620,14705800,💫☑🙏भारत के सुप्रसिद्ध { 📿ज्योतिष तांत्रिक .+91...,NaN,NaN,NaN,NaN
4433540,2021-06-01 11:12:32,https://www.facebook.com/groups/38576731905413...,-1.611111,लव-डाँडो,385767319054135,14621639,भन्नेले जे भनुन 😱😱 तिमी मेरै त हौ नि मेरो हुने...,NaN,NaN,NaN,NaN
3483596,2021-05-19 17:59:21,https://www.facebook.com/groups/38478147242457...,-19.000000,"हिंदुस्तान के उत्तराधिकारी ""योगी आदित्यनाथ""",384781472424574,14621350,यास तूफान बंगाल की खाड़ी एक और जागरूकता अभियान...,NaN,NaN,NaN,NaN
3215681,2021-04-02 02:53:59,https://www.facebook.com/groups/27040343599220...,-11.000000,नीम करोली बाबा,2704034359922038,17573129,*यो ध्रुवाणि परित्यज्य ह्यध्रुवं परिसेवते।* *ध...,NaN,NaN,NaN,NaN
2204412,2021-03-22 13:05:03,https://www.facebook.com/groups/49399864594047...,-104.000000,इन्द्रेणी समाज Indreni Samaj✅,4939986459404758,14876795,माधव नेपाललाई सोधिएको का'र'वा'ही'को पत्रमा यस्...,माधव नेपाललाई सोधिएको का’र’वा’ही’को पत्रमा यस्...,माधव नेपाललाई सोधिएको का’र’वा’ही’को पत्रमा यस्...,https://sewakhabar.com/26562/,sewakhabar.com


In [13]:
import re
import string
import warnings
from bs4 import BeautifulSoup

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)




remove_urls = lambda x: re.sub("http(.+)?(\W|$)", ' ', x)
remove_RT = lambda x: x.replace("RT ", "")
remove_mentions = lambda x: re.sub("@\S+", '', x)
remove_hashtags = lambda x: re.sub("#\S+", '', x)
remove_digits = lambda x: re.sub("\d+", "", x)
remove_punct = lambda x: re.sub("!|\||\%|\.|\-|\/|:|…|,|\?|।+|'|⁉|\*|‘|’|\"|\(|\)+", "", x)
remove_emojis = lambda x: emoji_pattern.sub("", x)
normalize_spaces = lambda x: re.sub("[\n\r\t ]+", ' ', x)


In [14]:
remove_noise = lambda x:normalize_spaces(
                                remove_emojis(
                                    remove_punct(
                                        remove_digits(
                                            remove_hashtags(
                                                remove_mentions(
                                                    remove_punct(
                                                        remove_urls(x.lower()))))))))


In [15]:
fdata['clean_text'] = fdata['message'].parallel_apply(lambda x: remove_noise(x) if len(x) >15 else '0')

In [16]:
fdata.head()

,date,postUrl,score,handle,platformId,id,message,title,description,link,caption,clean_text
1,2021-01-31 23:59:51,https://www.facebook.com/groups/22292669671263...,-14.000000,मैं मोदी और योगी आदित्यनाथ का सार्थक हूं,2229266967126315,14848753,Jay hind jay mahadev jay modi ji,NaN,NaN,NaN,NaN,jay hind jay mahadev jay modi ji
2,2021-01-31 23:59:46,https://www.facebook.com/groups/74054456632282...,5.713333,हिंदुत्व सम्राट योगी आदित्यनाथ ✅,740544566322820,8951434,सुबह के 5:30 बज रहे हैं जो भी भाई बहन उठ गये ह...,NaN,NaN,NaN,NaN,सुबह के बज रहे हैं जो भी भाई बहन उठ गये है वह ...
6,2021-01-31 23:58:08,https://www.facebook.com/groups/42729257728543...,-1.177215,"भगवानको भक्त बनौ आस्था जगाऔ""",427292577285439,14632578,ॐ ॐ ॐ जय शम्भो आज सोमबार भगवfन शिव को दर्शन गर...,आजको राशिफल बि.सं. २०७७ साल माघ १९ गते सोमबार ...,आजको राशिफल बि.सं. २०७७ साल माघ १९ गते सोमबार ...,https://nepaljyotish.com/daily-rashifal-nepali...,nepaljyotish.com,ॐ ॐ ॐ जय शम्भो आज सोमबार भगवfन शिव को दर्शन गर...
14,2021-01-31 23:55:40,https://www.facebook.com/groups/13687590455119...,-62.000000,भारतीय किसान आंदोलन,136875904551197,14541307,इतिहास दोहराया जा रहा है; जो लोग अंग्रेजो से न...,NaN,NaN,NaN,NaN,इतिहास दोहराया जा रहा है; जो लोग अंग्रेजो से न...
16,2021-01-31 23:55:27,https://www.facebook.com/groups/66920015679729...,7.104520,बाबा श्याम के दीवाने,669200156797296,8809601,5:30 बजने वाले हैं अगर जाग रहे हो तो अपनी अपनी...,NaN,NaN,NaN,NaN,बजने वाले हैं अगर जाग रहे हो तो अपनी अपनी हाज...


In [17]:
frame = fdata.copy()

In [18]:
print(len(frame))
frame = frame.loc[frame['clean_text']!='0']
print(len(frame))

1982899
1893619


In [19]:
frame['textlen'] = frame['clean_text'].parallel_apply(lambda x: len(x.split(" ")))

In [20]:
frame = frame.loc[(frame['textlen']>14)&(frame['textlen']<240)]
len(frame)

1128583

In [21]:
frame = frame.drop_duplicates()
len(frame)

1128583

In [22]:
print(Counter(frame['handle'].tolist()))

Counter({'किसान आईटी सेल/Farmer Itcell/ਕਿਸਾਨ ਆਈਟੀ ਸੈੱਲ': 29352, 'भारतीय किसान आंदोलन': 16922, 'हँसो और हँसाओ': 12654, 'भारतीय किसान आंदोलन से जुड़िए।': 12347, 'कविताएं डॉ अंशु के साथ': 10922, 'विश्व हिंदी संस्थान, कनाडा': 10317, 'आजको शिक्षा खबर तथा शैक्षिक जानकारी': 10306, 'डॉक्टर सुरेन्द्र केसी समर्थकहरू DOCTOR SURENDRA KC FANS CLUB': 10286, 'SHARE BAZAR NEWS { सेयर बजार,सेयर सम्बन्धिका बिचार }': 10018, 'काव्योदय': 9538, 'इन्द्रेणी समाज Indreni Samaj✅': 7854, 'ज्योतिष ,मंत्र ,तंत्र ,यन्त्र एवम नवरतन कलश': 6871, 'श्याम परिवार पहाड़ गंज': 6835, 'भारतीय राष्ट्रीय कांग्रेस ,आई टी सेल': 6030, 'लड़ेंगे जीतेंगे (साथ देने के लिए 100 लोगों को जोड़ें)': 5992, 'लव-डाँडो': 5954, '🚩 कट्टर मोदी समर्थक ग्रुप से जुड़ते ही अपने 51 साथियो को भी  जोड़े ।🚩': 5541, 'राजा ज्ञानेन्द्र - King Gyanendra (राष्ट्रभक्त राजा )': 5466, 'REET 31000, सपने जो सच होंगे L 1-2': 5424, '🙏🙏"हमारा परिवार "🙏🙏': 5397, 'Boycott Modia - इस मिशन से जुड़ने के लिए अपने 100 दोस्तों को ऐड करे': 4871, 'अल्फ़ाज़ सिर्फ़ आपके ❤️': 4849, '💖""

In [23]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('distiluse-base-multilingual-cased-v2')

In [65]:
frame = frame.loc[frame['title']!='This is a re-share of a post']
frame = frame.drop_duplicates(subset=['handle','message'])
print(len(frame))

389996


In [98]:
sample_frame = frame.sample(15000)
sample_text = sample_frame['clean_text'].tolist()
embeddings = embedder.encode(sample_text)
sample_cosSim = cosine_similarity(embeddings)
np.fill_diagonal(sample_cosSim, 0)

In [99]:
sample_cosSim.shape

(15000, 15000)

In [100]:
embeddings.shape

(15000, 512)

In [101]:
import hdbscan

In [102]:
sample_cosSim[np.isnan(sample_cosSim)] = 0
sample_cosSim[np.isinf(sample_cosSim)] = 0


In [103]:
sample_cosSim.dtype

dtype('float32')

In [104]:
sample_cosSim = sample_cosSim.astype('double')
sample_cosSim.dtype

dtype('float64')

In [105]:
sample_codDis = 1 - sample_cosSim

In [106]:
sample_codDis

array([[1.        , 0.88166577, 0.98991075, ..., 0.8284384 , 0.60142371,
        1.04958021],
       [0.88166577, 1.        , 0.90293115, ..., 0.86028057, 0.73642373,
        0.93000396],
       [0.98991075, 0.90293115, 1.        , ..., 0.77515593, 0.84500769,
        1.05253049],
       ...,
       [0.8284384 , 0.86028057, 0.77515593, ..., 1.        , 0.89515705,
        1.0438769 ],
       [0.60142371, 0.73642373, 0.84500769, ..., 0.89515705, 1.        ,
        0.93637782],
       [1.04958021, 0.93000396, 1.05253049, ..., 1.0438769 , 0.93637782,
        1.        ]])

In [107]:
#sample_codDis[sample_codDis>0.2] = np.inf

In [108]:
clusterer = hdbscan.HDBSCAN(metric='precomputed', min_cluster_size=3, core_dist_n_jobs=100, alpha=10.)
clusterer.fit(sample_codDis)
labels = clusterer.labels_

In [109]:
labels

array([-1, -1, -1, ..., -1, -1, -1])

In [110]:
sample_frame['clutser_labels'] = labels
print(Counter(sample_frame['clutser_labels'].tolist()))

Counter({-1: 14921, 13: 22, 12: 8, 11: 7, 9: 7, 7: 6, 1: 4, 8: 4, 3: 3, 10: 3, 6: 3, 4: 3, 0: 3, 5: 3, 2: 3})


In [122]:
sample_frame.loc[sample_frame['clutser_labels']==1]#['postUrl'].tolist()

,date,postUrl,score,handle,platformId,id,message,title,description,link,caption,clean_text,textlen,clutser_labels
1478727,2021-02-23 17:50:40,https://www.facebook.com/groups/27717355829549...,-0.352941,जमींदार एकता मंच,2771735582954990,14866093,आसाराम अकेला ऐसा बदनसीब है 😐 जो समय रहते बीजेप...,NaN,NaN,NaN,NaN,आसाराम अकेला ऐसा बदनसीब है जो समय रहते बीजेपी ...,21,1
523694,2021-01-11 02:16:47,https://www.facebook.com/groups/29741149803043...,-14.333333,किसान आईटी सेल/Farmer Itcell/ਕਿਸਾਨ ਆਈਟੀ ਸੈੱਲ,297411498030431,14594928,"""देशभक्ति का मतलब देश के साथ खड़ा होना होता है...",NaN,NaN,NaN,NaN,देशभक्ति का मतलब देश के साथ खड़ा होना होता है ...,20,1
1521125,2021-02-22 01:51:18,https://www.facebook.com/groups/36683207756679...,-14.800000,किसान मजदूर एकता,366832077566791,14616363,आसाराम अकेला ऐसा बदनसीब है 😐 जो समय रहते बीजेप...,NaN,NaN,NaN,NaN,आसाराम अकेला ऐसा बदनसीब है जो समय रहते बीजेपी ...,21,1
436959,2021-01-14 23:42:35,https://www.facebook.com/groups/15992549570353...,3.693878,"SHARE BAZAR NEWS { सेयर बजार,सेयर सम्बन्धिका ब...",1599254957035341,10152917,शुक्रबार पनि सेयर कारोबार सुचारु गरिनु पर्छ । ...,NaN,NaN,NaN,NaN,शुक्रबार पनि सेयर कारोबार सुचारु गरिनु पर्छ सव...,21,1


In [120]:
sample_ids = sample_frame['id'].tolist()
i_1 = sample_ids.index(14594928)
i_2 = sample_ids.index(10152917)
sample_codDis[i_1, i_2]

0.9237479344010353

In [33]:
good_matches = []
for i in range(100):

    test_indice = sample(list(range(10000)), 1)[0]
    
    max_id = np.argmax(sample_cosSim[test_indice, :])
    if sample_cosSim[test_indice, max_id] > 0.85 and sample_cosSim[test_indice, max_id]<0.95:
        good_matches.append((test_indice, max_id, sample_cosSim[test_indice, max_id]))
#         print(sample_cosSim[test_indice, max_id])
#         sample_frame.iloc[[test_indice, max_id], :]
#         print(sample_text[test_indice])
#         print(sample_cosSim[test_indice, max_id])
#         print("\n")
#         print(sample_text[max_id])
#         print("-------------------------------------------------------------------")

In [34]:
good_matches

[(2895, 4918, 0.91264755),
 (7529, 519, 0.9404557),
 (1080, 5434, 0.8574138),
 (3711, 4351, 0.9453677),
 (6664, 668, 0.9200531),
 (5251, 7753, 0.9197302),
 (5986, 6028, 0.87085235)]

In [35]:
E = sample(range(len(good_matches)), 1)[0]
pd.set_option("max_colwidth", -1)
sample_frame.iloc[[good_matches[E][0], good_matches[E][1]], :]

<ipython-input-35-fcfab733b14d>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("max_colwidth", -1)


,date,postUrl,score,handle,platformId,id,message,title,description,link,caption,clean_text,textlen
2581,2021-01-31 17:24:18,https://www.facebook.com/groups/2229492304031586/permalink/2749726968674781,19.304348,🚩🚩अगर आप कट्टर राजपूत है join करें हिन्दुस्तान की सान,2229492304031586,14848764,स्मरण रहे राम मंदिर हमारे दिए नोट से नहीं हमारे दिए वोट‌ से बन रहा है। 🚩जय श्री राम🚩,NaN,NaN,NaN,NaN,स्मरण रहे राम मंदिर हमारे दिए नोट से नहीं हमारे दिए वोट‌ से बन रहा है जय श्री राम,19
19156,2021-01-31 07:11:59,https://www.facebook.com/groups/317217575656189/permalink/721515178559758,-2.797101,पूछता है भारत ®️ ☑,317217575656189,9053966,"Never forget 👇 श्री राम मंदिर आपके नोटों से नहीं,आपके वोटों से बन रहा है। 🚩जय श्री राम🚩🏹",NaN,NaN,NaN,NaN,never forget श्री राम मंदिर आपके नोटों से नहींआपके वोटों से बन रहा है जय श्री राम,17


In [182]:
import requests, urllib3, urllib

html_ = urllib.request.urlopen("https://www.facebook.com/groups/1865102653795228/permalink/2539278413044312")
#.get("https://www.facebook.com/groups/1865102653795228/permalink/2539278413044312")
strresponse = html_.read()

In [183]:
strresponse

b'<!DOCTYPE html><html id="facebook" class="_9dls" lang="en" dir="ltr"><head><link data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/ye/r/Ta8_J_nYekI.ico" rel="shortcut icon" href="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" /><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" /><link rel="alternate" hreflang="x-default" href="https://www.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="ar" href="https://ar-ar.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="bg" href="https://bg-bg.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="bn" href="https://bn-in.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="bs" href="https://bs-ba.facebook.com/groups/186510265379

In [185]:
strresponse.decode('utf-8')

'<!DOCTYPE html><html id="facebook" class="_9dls" lang="en" dir="ltr"><head><link data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/ye/r/Ta8_J_nYekI.ico" rel="shortcut icon" href="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" /><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" /><link rel="alternate" hreflang="x-default" href="https://www.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="ar" href="https://ar-ar.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="bg" href="https://bg-bg.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="bn" href="https://bn-in.facebook.com/groups/1865102653795228/posts/2539278413044312/" /><link rel="alternate" hreflang="bs" href="https://bs-ba.facebook.com/groups/1865102653795

In [137]:
from bs4 import BeautifulSoup

In [187]:
soup = BeautifulSoup(strresponse.decode("utf-8"), 'html.parser')

In [193]:
soup.prettify()

'<!DOCTYPE html>\n<html class="_9dls" dir="ltr" id="facebook" lang="en">\n <head>\n  <link data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/ye/r/Ta8_J_nYekI.ico" data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" href="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" rel="shortcut icon"/>\n  <meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/>\n  <link href="https://www.facebook.com/groups/1865102653795228/posts/2539278413044312/" hreflang="x-default" rel="alternate"/>\n  <link href="https://ar-ar.facebook.com/groups/1865102653795228/posts/2539278413044312/" hreflang="ar" rel="alternate"/>\n  <link href="https://bg-bg.facebook.com/groups/1865102653795228/posts/2539278413044312/" hreflang="bg" rel="alternate"/>\n  <link href="https://bn-in.facebook.com/groups/1865102653795228/posts/2539278413044312/" hreflang="bn" rel="alternate"/>\n  <link href="https://bs-ba.facebook.com/groups/1865102653795228